In [3]:
#;.pykx.disableJupyter()

PyKX now running in 'python' mode (default). All cells by default will be run as python code. 
Include '%%q' at the beginning of each cell to run as q code. 


In [4]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


In [5]:
trade:([]time: 10000?.z.n; sym:10000?`JPM`GE`MSFT;size:10000?1000;price:10000?12.0;side:10000?`B`S;exchange:10000?`N`T`L);
trade:update asc time from trade;

quote: delete price, side, exchange from update sym: 10000?`JPM`GE`MSFT, time:asc 10000?.z.n, ask:price+10000?0.5, bid: price - 10000?0.55 from trade;

**Learning Outcomes**

To understand:
* What are functional statements?
* When to use a functional statement
* Functional Parse trees
* How to identify a functional statement 
* Functional forms of select, update, exec and delete


# Introduction
[Functional qSQL](https://code.kx.com/q/basics/funsql/) statements are the functional forms associated with the qSQL queries that we are familiar with from the Queries module - namely `select`,`exec`,`delete` and `update`. They are particularly useful for generating dynamic queries, such as specifying particular column names for retrieval. 

|qSql query | Syntax | Purpose |
|-------------|-------|-------|
|select | ?[table;constraints;grouping;return cols] | returning a subset of a table in a table format|
|select | ?[table;constraints;grouping;return cols;n] |select up to n records |
|select | ?[table;constraints;grouping;return cols;n;(g;cn)] | select up to n records sorted by g on cn |
|exec | ?[table;constraints;grouping;return cols] | returning a subset of a table in a list or dictionary format| 
|exec | ?[table;index;return cols] | Can retrieve specific rows from a table- another form of Exec |
|update | ![table;constraints;grouping;return cols] | updating rows in a table |
|delete | ![table;constraints;grouping;return cols] | deleting rows from a table |

## Why use Functional qSQL?
Fundamentally, functional forms allow us to create our queries programmatically so we can respond to variable inputs. This in turn allows us to produce more dynamic outputs i.e. return different table schemas depending on the input provided or running the same query over multiple tables. Eg. Getting close prices from multiple data sources. 

Consider and discuss how you would do the below using functions and the qSQL statements you are already familiar with: 

 ##### Group Exercises 
1. How would we write a function that takes a list of columns and returns data from our `trade` table?


In [ ]:
returnCols:{[columns] to_select: $[1<count columns;","sv string columns;string columns];
                query:"select ",to_select," from trade";
                -1 "Our query: ",query;
                value query}

returnCols[`price`time`sym]


In [10]:
//your answer here 
//cols trade
5#select price, time, sym from trade

returnCols:{[columns] to_select: $[1<count columns;","sv string columns;string columns];
                query:"select ",to_select," from trade";
                -1 "Our query: ",query;
                value query}

returnCols[`price`time`sym]

price    time                 sym 
----------------------------------
2.612327 0D00:00:00.004375310 JPM 
8.989    0D00:00:00.046089480 GE  
5.550662 0D00:00:00.144635306 MSFT
7.389787 0D00:00:00.165940544 JPM 
3.557074 0D00:00:00.175220678 MSFT
price    time                 sym 
----------------------------------
2.612327 0D00:00:00.004375310 JPM 
8.989    0D00:00:00.046089480 GE  
5.550662 0D00:00:00.144635306 MSFT
7.389787 0D00:00:00.165940544 JPM 
3.557074 0D00:00:00.175220678 MSFT
7.154379 0D00:00:00.182020766 JPM 
9.207793 0D00:00:00.236256852 JPM 
9.338188 0D00:00:00.245483237 JPM 
6.18538  0D00:00:00.262837919 GE  
9.724386 0D00:00:00.267904380 MSFT
7.547554 0D00:00:00.275574979 JPM 
2.215053 0D00:00:00.277730277 JPM 
3.585405 0D00:00:00.284491743 JPM 
9.465283 0D00:00:00.285450072 GE  
3.626004 0D00:00:00.295165534 GE  
9.554201 0D00:00:00.310365260 JPM 
10.72355 0D00:00:00.323367932 JPM 
6.345187 0D00:00:00.341562921 GE  
4.040299 0D00:00:00.351501211 GE  
2.552804 0D00:00:00.

2. How would we modify that function to allow users to define aggregations for each column? e.g. average price, min size etc.

In [ ]:
returnColswAggs:{[columns;aggregations] 
                pairs: aggregations,'columns;
                metrics: " " sv' string pairs; 
                colNames: `$"_" sv' string pairs;
                to_select: $[1<count metrics;"," sv metrics; metrics];
                query:"select ",to_select," from trade";
                -1 "Our query: `",("`" sv string[colNames])," xcol ",query;
                colNames xcol value query}

returnColswAggs[`price`size;(avg;max)]

In [9]:
//your answer here 

returnColswAggs:{[columns;aggregations] 
                pairs: aggregations,'columns;
                metrics: " " sv' string pairs; 
                colNames: `$"_" sv' string pairs;
                to_select: $[1<count metrics;"," sv metrics; metrics];
                query:"select ",to_select," from trade";
                -1 "Our query: `",("`" sv string[colNames])," xcol ",query;
                colNames xcol value query}

returnColswAggs[`price`size;(avg;max)]

avg_price max_size
------------------
5.995963  999     
Our query: `avg_price`max_size xcol select avg price,max size from trade


## Refresher: Functional forms and the `parse` tree
Before we look at how to construct a functional statement, we will discuss functional forms and specifically the keyword [`parse`](https://code.kx.com/v2/ref/parse/#q-sql) which allows us view any q statement in it's functional form.

`parse` takes a string as input and returns the corresponding "parse tree", which is what the compiler uses in order to execute the code. These can be evaluated using [`value`](https://code.kx.com/q/ref/value/) or [`eval`](https://code.kx.com/q/ref/eval/).

In [11]:
+[2;2]                               //2+2
wavg[10 10 20;2 3 1]                 //10 10 20 wavg 2 3 1 

4
1.75


In [14]:
value parse "2+2"
(+;2;2)~parse "2+2"

4
1b


In [13]:
value (+;2;2)
eval (+;2;2)

4
4


Functional form is a listing of operators and their arguments in **implementation order**: 

In [17]:
parse"2*4-2% -2# 1 2 3 4 5"

*
2
(-;4;(%;2;(#;-2;1 2 3 4 5)))


So looking at the second line of the above, and starting from the inside out we have `(#;-2;1 2 3 4 5)`. This is at the deepest level of nesting and so is performed first. The result of that is then used with `%` to divide two and so the operation continues.  

##### Exercise 

Write the parse trees for the following commands:

* (7*2+3)%2 [returns 17.5]

Each should return the value as indicated when evaluated with `eval`. 

In [ ]:
eval(%;(*;7;(+;2;3));2)  //last operation is the division, after that *, then + 
eval parse"(7*2+3)%2"

In [19]:
/your answer here 
parse "(7*2+3)%2"

%
(*;7;(+;2;3))
2


Can you write these parse trees as standard q expressions? (evaluating the parse should match your answer) 
* (wavg;1 2 3;(til;3))
* ((';,);"abc";1 2 3) 

In [24]:
1 2 3 wavg til 3
eval (wavg;1 2 3;(til;3))

1.333333
1.333333


In [ ]:
"abc",'1 2 3
eval ((';,);"abc";1 2 3) //we can see here that an iterator is a function that takes as input another function 

In [23]:
//your answer here 
1 2 3 wavg til 3

"abc", '1 2 3 

1.333333
"a" 1
"b" 2
"c" 3


# Basic Functional form of qSQL 
Each qSQL query is one of four types of commands - either `select`,`exec`,`update` or `delete`. They all conform to the format we've seen above, where for example the first input is always table, the second always related to the where clause etc. However, the types of the inputs for the by clause and return clause differ between forms. 

Basic Form structure follows every other function in kdb+: 

    query_function[tableToQuery;whereClause;byClause;returnClause]
   
* `query_function` - A `select` or `exec` query uses the `?` operator. A `delete` and `update` statement uses the `!` operator
* `tableToQuery` - The table to query
* `whereClause` - Any filters to be applied to the table are included here as a list of functional forms
* `byClause` - Any groupings to be applied 
* `returnClause`- The columns to be returned and any operations to be performed on them


##### Exercise
Convert the qSQL statement `"select from trade"` to the equivalent functional form and verify these match. (Hint: Use `parse`)

In [ ]:
parse "select from trade"

In [ ]:
?[trade;();0b;()]
?[trade;();0b;()]~select from trade  //congratulations! you just wrote you first functional select! 

In [29]:
//your answer here 
?[trade;();0b;()]
value parse "select from trade"

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.004375310 JPM  273  2.612327 B    N       
0D00:00:00.046089480 GE   16   8.989    S    N       
0D00:00:00.144635306 MSFT 57   5.550662 B    N       
0D00:00:00.165940544 JPM  606  7.389787 S    T       
0D00:00:00.175220678 MSFT 288  3.557074 S    T       
0D00:00:00.182020766 JPM  101  7.154379 B    T       
0D00:00:00.236256852 JPM  508  9.207793 B    N       
0D00:00:00.245483237 JPM  177  9.338188 B    N       
0D00:00:00.262837919 GE   879  6.18538  B    L       
0D00:00:00.267904380 MSFT 131  9.724386 B    N       
0D00:00:00.275574979 JPM  395  7.547554 S    T       
0D00:00:00.277730277 JPM  676  2.215053 B    L       
0D00:00:00.284491743 JPM  467  3.585405 S    N       
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:00.295165534 GE   371  3.626004 S    T       
0D00:00:00.310365260 JPM  453  9.554201 B    N       
0D00:00:00.323367932 JPM  79

Convert the qSQL statement `"exec from trade"` to the equivalent functional form and verify these match. 

In [ ]:
parse "exec from trade"

In [ ]:
?[trade;();();()]

In [32]:
//your answer here 
?[trade;();();()]
eval parse "exec from trade"

time    | 0D00:03:30.246117086
sym     | `MSFT
size    | 714
price   | 7.733435
side    | `B
exchange| `L
time    | 0D00:03:30.246117086
sym     | `MSFT
size    | 714
price   | 7.733435
side    | `B
exchange| `L


In [33]:
exec from trade

time    | 0D00:03:30.246117086
sym     | `MSFT
size    | 714
price   | 7.733435
side    | `B
exchange| `L


What (if any) difference did you notice between the `exec` and `select` statements?

The third parameter in the statements differs! Is is actually the type of the third argument that determines if the statement is to be interpreted as either a `select` or `exec` query.

In [ ]:
//your answer here 
//The type of the third entry differs, and that then determines whether we have a select statement or an exec statement.

# Functional `select`
In this section we will discuss how to write some `select` qSQL queries in their functional form and provide a detailed overview of the inputs to the functional form.

We have already seen the most simple implementation of a functional select: 

In [ ]:
select from trade        //selecting all columns with no constraints or groupingretffg

In [ ]:
parse"select from trade" //functional form

In [ ]:
t:trade
whereClause:()                     //there are no constraints
byClause:0b                        //there are no groupings 
returnClause:()                    //no specific column returns required
?[t;whereClause;byClause;returnClause] 

## Constraints (or where) clause 
Constraints are provided as a list of parse trees - each parse tree corresponds to a where clause which will act as a filter on the table.

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> The constraints are applied in order of appearance and so it's important to order them from most restrictive to least! Always start with any partition clauses first. </i></p>

In [34]:
select from trade where size>998 //filtering by size

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:16.485706534 GE   999  5.431377 B    N       
0D00:00:45.917069091 JPM  999  9.891391 B    L       
0D00:01:18.988498302 JPM  999  6.174487 S    L       
0D00:01:38.668940517 JPM  999  2.613197 S    L       
0D00:01:50.559914232 GE   999  10.56602 B    T       
0D00:02:17.129404855 MSFT 999  6.893306 B    L       
0D00:02:33.706903582 MSFT 999  10.964   B    N       


We can first look at the `parse` tree: 

In [35]:
parse"select from trade where size>998" 

?
`trade
,,(>;`size;998)
0b
()


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> In the statement above we see an ever present <code>,</code> that accompanies the constraints clause of a functional query when a where clause is present. The secondary <code>,</code> indicates we need to <code>enlist</code> our where clause to conform to the expected format, a list of parse trees. </i></p>

In [36]:
//we also see this when we look at the output format for a list of size one 
`notAList
enlist `oneItemList //this has a leading comma to indicate it is a list and not a singular item

notAList
,`oneItemList


We can look at the parse tree of our where clauses individually for clarity and particularly if they are complex.

In [37]:
parse "size > 998"  

>
`size
998


Working from our previous statement, our `whereClause` argument must be amended so we can filter by size.

In [39]:
t:trade
whereClause:enlist(>;`size;998)     //parse tree of "size>998" - making it a list of parse trees
byClause:0b                         //there is no grouping
returnClause:()                     //no specific column returns required

//?[t;whereClause;byClause;returnClause] 
?[trade;enlist(>;`size;998);0b;()]

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:16.485706534 GE   999  5.431377 B    N       
0D00:00:45.917069091 JPM  999  9.891391 B    L       
0D00:01:18.988498302 JPM  999  6.174487 S    L       
0D00:01:38.668940517 JPM  999  2.613197 S    L       
0D00:01:50.559914232 GE   999  10.56602 B    T       
0D00:02:17.129404855 MSFT 999  6.893306 B    L       
0D00:02:33.706903582 MSFT 999  10.964   B    N       


If we add another constraint to our where clause we will not need to use `enlist` (as we will already be providing a list of parse trees).

 ##### Exercise 
Convert the qSQL statement below to a functional qSQL statement: 
    
    select from trade where size > 998, time < 09:00  

In [ ]:
//to begin we parse to see the functional form: 
parse "select from trade where size>998, time < 09:00"

In [ ]:
//adding the x
t:trade
whereClause:((>;`size;998);(<;`time;09:00))   
byClause:0b                         
returnClause:()

?[t;whereClause;byClause;returnClause] 

In [ ]:
select from trade where size>998, time < 09:00

In [43]:
//your answer here 
parse "select from trade where size > 998, time < 09:00 "

?
`trade
,((>;`size;998);(<;`time;09:00))
0b
()


In [50]:
?[trade;((>;`size;998);(<;`time;09:00));0b;()]

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:16.485706534 GE   999  5.431377 B    N       
0D00:00:45.917069091 JPM  999  9.891391 B    L       
0D00:01:18.988498302 JPM  999  6.174487 S    L       
0D00:01:38.668940517 JPM  999  2.613197 S    L       
0D00:01:50.559914232 GE   999  10.56602 B    T       
0D00:02:17.129404855 MSFT 999  6.893306 B    L       
0D00:02:33.706903582 MSFT 999  10.964   B    N       


### Constraints using symbols

Let's now look at an example where we want to use a symbol within our query constraint:

In [51]:
parse "select from trade where sym=`JPM,size>998"

?
`trade
,((=;`sym;,`JPM);(>;`size;998))
0b
()


Here we can see another `,` appearing, this time before <code>\`JPM</code>!

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Within parse trees, we need a way to distinguish between symbols as a type and symbolic variable names to be evaluated. We enlist when we want the statement to interpret symbols as a literal rather than as a variable. </i></p>

In [52]:
t:trade
whereClause:((=;`sym;enlist `JPM);(>;`size;998))   
byClause:0b                         
returnClause:() 

?[t;whereClause;byClause;returnClause] 

time                 sym size price    side exchange
----------------------------------------------------
0D00:00:45.917069091 JPM 999  9.891391 B    L       
0D00:01:18.988498302 JPM 999  6.174487 S    L       
0D00:01:38.668940517 JPM 999  2.613197 S    L       


Notice the difference in the below statement where we instead use a reference to a variable we have defined:

In [89]:
myVariable:`JPM
parse "select from trade where sym=`JPM,size>998"

?
`trade
,((=;`sym;,`JPM);(>;`size;998))
0b
()


In [54]:
t:trade
whereClause:((=;`sym;`myVariable);(>;`size;998))     //without the enlist, the symbol is interpreted as a variable
byClause:0b                         
returnClause:() 

?[t;whereClause;byClause;returnClause]

time                 sym size price    side exchange
----------------------------------------------------
0D00:00:45.917069091 JPM 999  9.891391 B    L       
0D00:01:18.988498302 JPM 999  6.174487 S    L       
0D00:01:38.668940517 JPM 999  2.613197 S    L       


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> When kdb+/q is looking for a variable, it first "looks" for it within the table scope, next in the scope of the function (if called within a function), finally it will look in the global scope. If kdb+/q can't resolve the variable, it will repeat it back to you as an error! Further detail and examples of this in the <b>Practical Guidance</b> notebook.</i></p>

It can be tricky sometimes to figure out these statements, but this will definitely come with practice! Thankfully we can use `parse` as a cheat function to help us form these statements in the meantime.

##### Exercise

Write a functional `select` statement using the table `trade`  -  apply the following filters:
* The sym is either <code>\`JPM</code> or <code>\`MSFT</code> 
* The trade size is less than 200
* The price is between 4 and 10

In [76]:
//we'll start from our parse tree and then work back!
parse"select from trade where sym in `JPM`MSFT, size<200, price within 4 10" 

?
`trade
,((in;`sym;,`JPM`MSFT);(<;`size;200);(within;`price;4 10))
0b
()


In [87]:
c:enlist (in;`sym;enlist`JPM`MSFT)  //our first clause needs to enlist the symbols as they are literal values
show c,:(<;`size;200)                    //appending to our constraint clause - straightforward parse tree from "size<200"
//c,:(within;`price;(4;10))           //nested list for within's second input - (within;`price;4 10) works too
//c                                   //printing our constraint
//?[trade;c;0b;()]                    //no aggregations or groupings - return table with filters applied

in `sym  ,`JPM`MSFT
<  `size 200       


In [88]:
//your answer here 

parse "select from trade where sym in `JPM`MSFT, size>200, price within 4 10"
//show variable:`JPM`MSFT
//select from trade where sym in variable, size<200, price within 4 10
?[trade;((in;`sym;enlist`JPM`MSFT);(<;`size;200);(within;`price;(4 10)));0b;()]

?
`trade
,((in;`sym;,`JPM`MSFT);(>;`size;200);(within;`price;4 10))
0b
()
time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.144635306 MSFT 57   5.550662 B    N       
0D00:00:00.182020766 JPM  101  7.154379 B    T       
0D00:00:00.245483237 JPM  177  9.338188 B    N       
0D00:00:00.267904380 MSFT 131  9.724386 B    N       
0D00:00:00.475220054 MSFT 199  8.938436 S    N       
0D00:00:00.559789508 MSFT 169  6.31297  B    T       
0D00:00:00.751196438 MSFT 18   4.34695  B    L       
0D00:00:01.766726027 JPM  181  8.44997  B    T       
0D00:00:02.498853541 JPM  190  5.333311 B    L       
0D00:00:02.677163615 JPM  115  6.576946 B    T       
0D00:00:04.307053042 JPM  148  8.281443 S    L       
0D00:00:04.596862516 MSFT 42   9.029184 S    L       
0D00:00:04.707308081 MSFT 158  7.710486 S    L       
0D00:00:04.813092751 MSFT 107  9.324027 B    T       
0D00:00:05.170807413 JPM  79   4.494317 B    N       
0D00:00:

In [61]:
f: `jpm

## Grouping (or by/group-by) clause  
This clause comes into play when we are grouping by certain column(s).

 ##### Syntax
The form that the `by` clause takes for a functional `select` is a dictionary. 

The **keys** of the dictionary are the **names** of the grouping columns that will form your resultant key or groups. 
The **values** associated with these keys are **parse trees** that will be evaluated to form our group data levels.

 ##### Example 
     
     byClause:`SYM`SIDE!`sym`side        
The returned table be keyed on our groupings with column names `SYM` and `SIDE`.

Let's consider the case where we want the last result for each `sym`- we can use the following qSQL statement: 

In [95]:
select by sym from trade 

sym | time                 size price    side exchange
----| ------------------------------------------------
GE  | 0D00:03:29.966030368 414  11.43102 B    L       
JPM | 0D00:03:30.229728517 868  4.874444 S    L       
MSFT| 0D00:03:30.246117086 714  7.733435 B    L       


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If you haven't seen this before - this is a neat overload of the by clause in a select statement! When we don't specify the data we would like returned, kdb+/q will return to us the last values in the table for each distinct combination in our <code>by</code> clause.</i></p>

Looking at the corresponding parse tree: 

In [96]:
parse"select by sym,exchange from trade" 

?
`trade
()
`sym`exchange!`sym`exchange
()


In [97]:
t:trade; 
whereClause:()
returnClause:()
byClause: `sym`exchange!`sym`exchange

?[t;whereClause;byClause;returnClause]                    //returning our table grouped by sym

sym  exchange| time                 size price    side
-------------| ---------------------------------------
GE   L       | 0D00:03:29.966030368 414  11.43102 B   
GE   N       | 0D00:03:29.759437369 507  8.900954 B   
GE   T       | 0D00:03:29.938758053 996  6.816952 B   
JPM  L       | 0D00:03:30.229728517 868  4.874444 S   
JPM  N       | 0D00:03:30.211511745 726  5.203375 B   
JPM  T       | 0D00:03:29.681444050 707  8.213949 B   
MSFT L       | 0D00:03:30.246117086 714  7.733435 B   
MSFT N       | 0D00:03:30.146521587 691  6.695458 S   
MSFT T       | 0D00:03:29.902283243 823  4.822801 B   


In [98]:
byClause: enlist [`newColName]!enlist `sym    //changing the keyname changes the column name 
?[t;whereClause;byClause;returnClause]      

newColName| time                 sym  size price    side exchange
----------| -----------------------------------------------------
GE        | 0D00:03:29.966030368 GE   414  11.43102 B    L       
JPM       | 0D00:03:30.229728517 JPM  868  4.874444 S    L       
MSFT      | 0D00:03:30.246117086 MSFT 714  7.733435 B    L       


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If we want to change the name of a column in our grouping, we can rename the key of the dictionary used in our group-by constraint.</i></p>

In [99]:
byClause: `SYM`exchange!`sym`exchange  //adding more columns to breakdown by
?[t;whereClause;byClause;returnClause]      

SYM  exchange| time                 sym  size price    side
-------------| --------------------------------------------
GE   L       | 0D00:03:29.966030368 GE   414  11.43102 B   
GE   N       | 0D00:03:29.759437369 GE   507  8.900954 B   
GE   T       | 0D00:03:29.938758053 GE   996  6.816952 B   
JPM  L       | 0D00:03:30.229728517 JPM  868  4.874444 S   
JPM  N       | 0D00:03:30.211511745 JPM  726  5.203375 B   
JPM  T       | 0D00:03:29.681444050 JPM  707  8.213949 B   
MSFT L       | 0D00:03:30.246117086 MSFT 714  7.733435 B   
MSFT N       | 0D00:03:30.146521587 MSFT 691  6.695458 S   
MSFT T       | 0D00:03:29.902283243 MSFT 823  4.822801 B   


Given that we have a time column in our table, let's look at another common analysis - getting the last trade every X hours (here we will use 2) and refer to that time as the `twoHourlyTime`, in qSQL we would do the following:

In [102]:
/select by twoHourlyTime:0D02:00:00 xbar time from trade
eval parse"select by twoHourlyTime:0D02:00:00 xbar time from trade" //uncomment if you want to see the parse tree

twoHourlyTime       | time                 sym  size price    side exchange
--------------------| -----------------------------------------------------
0D00:00:00.000000000| 0D00:03:30.246117086 MSFT 714  7.733435 B    L       


To reproduce this in our functional form we pass our parse tree expression for `"0D02:00:00 xbar time"` and assign that to our new columns name: 

In [103]:
t:trade; 
whereClause:()
returnClause:()
byClause: enlist[`twoHourlyTime]!enlist (xbar;0D02:00:00;`time)   //our parse tree from the above

?[t;whereClause;byClause;returnClause]        

twoHourlyTime       | time                 sym  size price    side exchange
--------------------| -----------------------------------------------------
0D00:00:00.000000000| 0D00:03:30.246117086 MSFT 714  7.733435 B    L       


Ta-dah! Hopefully you're now beginning to see that functional forms just boil down to using parse trees in appropriate configurations. 

##### Exercise 

Write a function `groupTrade` which takes a single input of either a column, or a list of columns, and returns the table `trade` grouped by those columns. The name of the grouping columns in the final table should be modified to have "_grouped" at the end of their new column name. 

Corresponding SQL Statement example:
    
    select by sym_grouped:sym, side_grouped:side from trade

Test your function with the following inputs: 
* <code>\`sym\`side\`exchange</code>
* <code>\`sym</code>


In [ ]:
groupTrade:{[columns] cList: (),columns; //making columns a list, if it isn't already
    names:`$string[cList],\:"_grouped";  //adding "_grouped" to each column after stringing them, then back to symbol
    bclause:names!cList;                 //pairing the new column names to the column values
    ?[trade;();bclause;()]               //returning the grouped table - no filters or aggregations
 }

In [ ]:
groupTrade[`sym`side`exchange]   //testing

In [ ]:
groupTrade[`sym]                  //testing

In [105]:
//your answer here 
parse "select by sym_grouped:sym, side_grouped:side from trade"
eval parse "select by sym_grouped:sym, side_grouped:side from trade"

?
`trade
()
`sym_grouped`side_grouped!`sym`side
()
sym_grouped side_grouped| time                 sym  size price    side exchange
------------------------| -----------------------------------------------------
GE          B           | 0D00:03:29.966030368 GE   414  11.43102 B    L       
GE          S           | 0D00:03:29.575256849 GE   779  1.358713 S    N       
JPM         B           | 0D00:03:30.211511745 JPM  726  5.203375 B    N       
JPM         S           | 0D00:03:30.229728517 JPM  868  4.874444 S    L       
MSFT        B           | 0D00:03:30.246117086 MSFT 714  7.733435 B    L       
MSFT        S           | 0D00:03:30.146521587 MSFT 691  6.695458 S    N       


In [106]:
?[trade;();`sym_grouped`side_grouped!`sym`side;()]

sym_grouped side_grouped| time                 sym  size price    side exchange
------------------------| -----------------------------------------------------
GE          B           | 0D00:03:29.966030368 GE   414  11.43102 B    L       
GE          S           | 0D00:03:29.575256849 GE   779  1.358713 S    N       
JPM         B           | 0D00:03:30.211511745 JPM  726  5.203375 B    N       
JPM         S           | 0D00:03:30.229728517 JPM  868  4.874444 S    L       
MSFT        B           | 0D00:03:30.246117086 MSFT 714  7.733435 B    L       
MSFT        S           | 0D00:03:30.146521587 MSFT 691  6.695458 S    N       


## Return (or aggregations) clause
We now know how to write a functional select with a `by` and `where` clause, but very often (and particularly when we are using `by` statements) we also want to return aggregations of our columns e.g. `max price`.

 ##### Syntax

Similarly to our by clause, the format of our return clause is also a dictionary mapping column names to values. The keys correspond to the name of the column and the values are constructed via a parse tree.

Let's suppose we want to return the open, high, low and close prices for each symbol from our trade table - here's our corresponding qSQL statement:

In [108]:
parse "select open:first price, high:max price, low: min price, close: last price by sym from trade"

?
`trade
()
(,`sym)!,`sym
`open`high`low`close!((*:;`price);(max;`price);(min;`price);(last;`price))


In [111]:
//syntax reminder! ?[t;c;b;a] t=table, c=constraints, b=by dictionary, a=aggregation dictionary
t:trade
whereClause:()                            //no constraints 
byClause:enlist[`sym]!enlist[`sym]     //just sym in the by clause
returnClause:`open`high`low`close!((first;`price);(max;`price);(min;`price);(last;`price)) 

?[t;whereClause;byClause;returnClause]      

sym | open     high     low         close   
----| --------------------------------------
GE  | 8.989    11.99864 0.001972016 11.43102
JPM | 2.612327 11.99709 0.002563655 4.874444
MSFT| 5.550662 11.994   0.001792207 7.733435


In [112]:
//reminder! For code readability we can also write our dictionary pairwise, if you find the above difficult to read
returnClause :.[!] flip ((`open;(first;`price));
        (`high;(max;`price));
        (`low;(min;`price));
        (`close;(last;`price)))

?[t;whereClause;byClause;returnClause]      

sym | open     high     low         close   
----| --------------------------------------
GE  | 8.989    11.99864 0.001972016 11.43102
JPM | 2.612327 11.99709 0.002563655 4.874444
MSFT| 5.550662 11.994   0.001792207 7.733435


This is much more like the usage we would see in conjunction with a by clause! 
 
If we want to return all the group values, we just select the column itself without applying an aggregation:

In [113]:
returnClause:.[!] flip ((`open;(first;`price));
        (`high;(max;`price));
        (`low;(min;`price));
        (`close;(last;`price));
        (`prices;`price))              //note we changed the column name here for clarity 
    
?[t;whereClause;byClause;returnClause]      

sym | open     high     low         close    prices                          ..
----| -----------------------------------------------------------------------..
GE  | 8.989    11.99864 0.001972016 11.43102 8.989 6.18538 9.465283 3.626004 ..
JPM | 2.612327 11.99709 0.002563655 4.874444 2.612327 7.389787 7.154379 9.207..
MSFT| 5.550662 11.994   0.001792207 7.733435 5.550662 3.557074 9.724386 2.552..


This aggregation clause can also be used without a `by` clause to just return columns. 

In [114]:
returnClause:x!x:(3#cols trade) //using # to return the first 3 columns in our table
returnClause

?[trade;();0b;returnClause]

time| time
sym | sym
size| size
time                 sym  size
------------------------------
0D00:00:00.004375310 JPM  273 
0D00:00:00.046089480 GE   16  
0D00:00:00.144635306 MSFT 57  
0D00:00:00.165940544 JPM  606 
0D00:00:00.175220678 MSFT 288 
0D00:00:00.182020766 JPM  101 
0D00:00:00.236256852 JPM  508 
0D00:00:00.245483237 JPM  177 
0D00:00:00.262837919 GE   879 
0D00:00:00.267904380 MSFT 131 
0D00:00:00.275574979 JPM  395 
0D00:00:00.277730277 JPM  676 
0D00:00:00.284491743 JPM  467 
0D00:00:00.285450072 GE   999 
0D00:00:00.295165534 GE   371 
0D00:00:00.310365260 JPM  453 
0D00:00:00.323367932 JPM  798 
0D00:00:00.341562921 GE   757 
0D00:00:00.351501211 GE   882 
0D00:00:00.420723251 MSFT 321 
..


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If we pass 1b in the <a href="https://code.kx.com/q/basics/funsql/#select-distinct">by-clause field</a> , we can get the distinct values of our returned columns. It's a special case for <code>select distinct<code/></i></p>

In [ ]:
//syntax reminder! ?[t;c;b;a] t=table, c=constraints, b=by dictionary, a=aggregation dictionary
t:trade
whereClause:()                            //no constraints 
byClause:1b     //changing by-clause to be true boolean
returnClause:enlist[`sym]!enlist `sym  

?[t;whereClause;byClause;returnClause] //same as select distinct sym from trade 

##### Exercise 

Write a functional `select` to calculate the volume weighted average price (`vwapPrice`) broken down by symbol (`sym`) in 15 minute time intervals (`interval`) using our `trade` table. 

In [129]:
//what do we want to return? That will be our returnClause - volume weighted average price 
returnClause: enlist[`vwapPrice]!enlist[(wavg;`size;`price)]
//what are we breaking down by? - by symbol (sym) in 15 minute time intervals (interval)
byClause:`sym`interval!(`sym;(xbar;0D00:15:00;`time))
//what constraints do we have? 
whereClause:() //none! 

?[trade;whereClause;byClause;returnClause]

sym  interval            | vwapPrice
-------------------------| ---------
GE   0D00:00:00.000000000| 5.934205 
JPM  0D00:00:00.000000000| 6.014708 
MSFT 0D00:00:00.000000000| 5.950301 


In [146]:
//your answer here
parse "select vwapPrice:(size wavg price) by sym from trade"
//trade
select vwapPrice:(size wavg price) by sym, Interval:(0D00:15:00 xbar time) from trade
//?[trade;();(enlist`sym)!enlist`sym;(enlist`vwapPrice)!enlist(wavg;`size;`price)]

?
`trade
()
(,`sym)!,`sym
(,`vwapPrice)!,(wavg;`size;`price)
sym  Interval            | vwapPrice
-------------------------| ---------
GE   0D00:00:00.000000000| 5.934205 
JPM  0D00:00:00.000000000| 6.014708 
MSFT 0D00:00:00.000000000| 5.950301 


# Functional `exec` 

Functional [`exec`](https://code.kx.com/q/basics/funsql/#exec) is very similar to functional `select`, however we will see that the syntax for returning and grouping (by) clauses differ.

## Return Clause 
Looking at where they differ - one of the easiest cases is to return a single column:

In [138]:
exec sym from trade

`JPM`GE`MSFT`JPM`MSFT`JPM`JPM`JPM`GE`MSFT`JPM`JPM`JPM`GE`GE`JPM`JPM`GE`GE`MSF..


In [139]:
parse"exec sym from trade"

?
`trade
()
()
,`sym


To write the functional form of the above query, we can see above differ from what we know with `select`. Previously we used `0b` to indicate no grouping and this is now `()`. Similarly, our dictionary aggregation input has been replaced by a single symbol value. 

In [140]:
?[trade;();();`sym]    

`JPM`GE`MSFT`JPM`MSFT`JPM`JPM`JPM`GE`MSFT`JPM`JPM`JPM`GE`GE`JPM`JPM`GE`GE`MSF..


Excellent! We have the same result! 

You might wonder what would happen if we wanted to return more columns: 

In [ ]:
?[trade;();();`exchange`sym]   //we can't pass two columns

This is expected behaviour - you'll remember that we can't do this with an exec anyways! When we return two items from an `exec` we get a dictionary return: 

In [141]:
exec exchange, sym from trade 
parse"exec exchange, sym from trade"

exchange| N   N  N    T   T    T   N   N   L  N    T   L   N   N  T  N   N   ..
sym     | JPM GE MSFT JPM MSFT JPM JPM JPM GE MSFT JPM JPM JPM GE GE JPM JPM ..
?
`trade
()
()
`exchange`sym!`exchange`sym


We can actually use this even with a single output, but by providing the dictionary, we get a dictionary output returned: 

In [142]:
?[trade;();();`exchange`sym!`exchange`sym]

?[trade;();();enlist[`sym]!enlist `sym]     
//equivalent to:
exec sym:sym from trade

exchange| N   N  N    T   T    T   N   N   L  N    T   L   N   N  T  N   N   ..
sym     | JPM GE MSFT JPM MSFT JPM JPM JPM GE MSFT JPM JPM JPM GE GE JPM JPM ..
sym| JPM GE MSFT JPM MSFT JPM JPM JPM GE MSFT JPM JPM JPM GE GE JPM JPM GE GE..
sym| JPM GE MSFT JPM MSFT JPM JPM JPM GE MSFT JPM JPM JPM GE GE JPM JPM GE GE..


## By Clause
The secondary way in which our `exec` statements diverge from our functional `select` is in the by clause. 

Lets suppose we want the following output: 

In [143]:
exec sym by exchange from trade
parse"exec sym by exchange from trade"

L| `GE`JPM`GE`JPM`MSFT`GE`JPM`MSFT`JPM`MSFT`JPM`GE`JPM`GE`MSFT`GE`MSFT`JPM`JP..
N| `JPM`GE`MSFT`JPM`JPM`MSFT`JPM`GE`JPM`JPM`MSFT`MSFT`MSFT`GE`MSFT`MSFT`JPM`G..
T| `JPM`MSFT`JPM`JPM`GE`GE`GE`MSFT`MSFT`MSFT`MSFT`GE`GE`GE`MSFT`MSFT`MSFT`MSF..
?
`trade
()
,`exchange
,`sym


We can actually just supply a single symbol of the `exchange` value to use as our group-by clause:

In [144]:
show execute:?[trade;();`exchange;`sym]

L| `GE`JPM`GE`JPM`MSFT`GE`JPM`MSFT`JPM`MSFT`JPM`GE`JPM`GE`MSFT`GE`MSFT`JPM`JP..
N| `JPM`GE`MSFT`JPM`JPM`MSFT`JPM`GE`JPM`JPM`MSFT`MSFT`MSFT`GE`MSFT`MSFT`JPM`G..
T| `JPM`MSFT`JPM`JPM`GE`GE`GE`MSFT`MSFT`MSFT`MSFT`GE`GE`GE`MSFT`MSFT`MSFT`MSF..


As we continue to grow this `exec` statement and add more return columns and grouping columns, we will see we actually just approach a `select`! 

We are now returning tables rather than the dictionary or lists structures that we usually use exec for: 

In [147]:
exec sym, price by exchange, 0D00:03:00 xbar time from trade //a keyed table 

exchange time                | sym                                           ..
-----------------------------| ----------------------------------------------..
L        0D00:00:00.000000000| `GE`JPM`GE`JPM`MSFT`GE`JPM`MSFT`JPM`MSFT`JPM`G..
L        0D00:03:00.000000000| `JPM`JPM`JPM`JPM`MSFT`GE`JPM`GE`GE`MSFT`MSFT`M..
N        0D00:00:00.000000000| `JPM`GE`MSFT`JPM`JPM`MSFT`JPM`GE`JPM`JPM`MSFT`..
N        0D00:03:00.000000000| `JPM`GE`JPM`MSFT`JPM`MSFT`JPM`GE`JPM`JPM`JPM`G..
T        0D00:00:00.000000000| `JPM`MSFT`JPM`JPM`GE`GE`GE`MSFT`MSFT`MSFT`MSFT..
T        0D00:03:00.000000000| `JPM`GE`MSFT`JPM`GE`GE`MSFT`MSFT`MSFT`JPM`GE`M..


In [148]:
parse "exec sym, price by exchange, 0D00:03:00 xbar time from trade"

?
`trade
()
`exchange`time!(`exchange;(k){x*y div x:$[16h=abs[@x];"j"$x;x]};0D00:03:00.00..
`sym`price!`sym`price


You can see we have now migrated to a `select` statement and can check equivalence. 

In [149]:
(exec sym, price by exchange, 0D00:03:00 xbar time from trade)~            //exec statement
          select sym, price by exchange, 0D00:03:00 xbar time from trade   //equivalent select statement

1b


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>
<p style='color:#273a6e'><i> You can think of functional <code>exec</code> statements as being special overloads of functional <code>select</code> statements.</i></p>


##### Exercise 

Write a functional `exec` statement to return from `trade` all columns - except for sym - as a dictionary, only for trades where the sym was JPM.

In [156]:
//our return clause needs to be a dictionary as we are returning many columns 
returnClause:x!x:cols[trade] except `sym //excluding `sym
//we are not grouping by anything
byClause:() //we don't want this as a table, so we provide an empty list rather than a boolean
//we are limiting to `JPM sym so need a constraint clause
whereClause:enlist (=;`sym;enlist `JPM) //enlisting as `JPM is a literal

?[trade;whereClause;byClause;returnClause]

time    | 0D00:00:00.004375310 0D00:00:00.165940544 0D00:00:00.182020766 0D00..
size    | 273                  606                  101                  508 ..
price   | 2.612327             7.389787             7.154379             9.20..
side    | B                    S                    B                    B   ..
exchange| N                    T                    T                    N   ..


In [159]:
//your answer here
5#trade
//parse "exec time, size, price, side, exchange from trade where sym=`JPM "
?[trade;enlist(=;`sym;enlist`JPM);();`time`size`price`side`exchange!`time`size`price`side`exchange]

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.004375310 JPM  273  2.612327 B    N       
0D00:00:00.046089480 GE   16   8.989    S    N       
0D00:00:00.144635306 MSFT 57   5.550662 B    N       
0D00:00:00.165940544 JPM  606  7.389787 S    T       
0D00:00:00.175220678 MSFT 288  3.557074 S    T       
time    | 0D00:00:00.004375310 0D00:00:00.165940544 0D00:00:00.182020766 0D00..
size    | 273                  606                  101                  508 ..
price   | 2.612327             7.389787             7.154379             9.20..
side    | B                    S                    B                    B   ..
exchange| N                    T                    T                    N   ..


# Functional `update` 
A functional [`update`](https://code.kx.com/q/basics/funsql/#update) can be used to change cell values, or add new columns. The syntax and inputs are the same as functional select, however we use `!` instead of `?`.

Using the parse function to create a functional update, you can see that it has the same syntax:

In [160]:
parse"update size+3 from trade where sym=`USDGBP"

!
`trade
,,(=;`sym;,`USDGBP)
0b
(,`size)!,(+;`size;3)


In [161]:
![trade;enlist (=;`sym;enlist `JPM);0b;(enlist `size)!enlist (+;`size;3000)] //updating our size+3000

time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.004375310 JPM  3273 2.612327 B    N       
0D00:00:00.046089480 GE   16   8.989    S    N       
0D00:00:00.144635306 MSFT 57   5.550662 B    N       
0D00:00:00.165940544 JPM  3606 7.389787 S    T       
0D00:00:00.175220678 MSFT 288  3.557074 S    T       
0D00:00:00.182020766 JPM  3101 7.154379 B    T       
0D00:00:00.236256852 JPM  3508 9.207793 B    N       
0D00:00:00.245483237 JPM  3177 9.338188 B    N       
0D00:00:00.262837919 GE   879  6.18538  B    L       
0D00:00:00.267904380 MSFT 131  9.724386 B    N       
0D00:00:00.275574979 JPM  3395 7.547554 S    T       
0D00:00:00.277730277 JPM  3676 2.215053 B    L       
0D00:00:00.284491743 JPM  3467 3.585405 S    N       
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:00.295165534 GE   371  3.626004 S    T       
0D00:00:00.310365260 JPM  3453 9.554201 B    N       
0D00:00:00.323367932 JPM  37

In the above statement we are applying our `update` statement to the column `size`, only where the condition is met. 

The primary difference here is the return - for `update` we also include all our other original table columns and rows (after updating where applicable), while for `select` we only return the specified columns and only where the criteria is met. 

In [162]:
?[trade;enlist (=;`sym;enlist `JPM);0b;(enlist `size)!enlist (+;`size;3000)] //select our size+3000

size
----
3273
3606
3101
3508
3177
3395
3676
3467
3453
3798
3200
3412
3584
3338
3922
3950
3720
3816
3178
3288
..


All other syntax is exactly the same!

In [163]:
//select and update are the same, when we return the same columns with no constaint
?[trade;();0b;cols[trade]!cols[trade]]~![trade;();0b;cols[trade]!cols[trade]]

1b


##### Exercise 

Write a functional `update` to add the column date (with todays date) to the `trade` table. 

In [169]:
![trade;();0b;enlist[`date]!enlist .z.d]

time                 sym  size price    side exchange date      
----------------------------------------------------------------
0D00:00:00.004375310 JPM  273  2.612327 B    N        2025.03.08
0D00:00:00.046089480 GE   16   8.989    S    N        2025.03.08
0D00:00:00.144635306 MSFT 57   5.550662 B    N        2025.03.08
0D00:00:00.165940544 JPM  606  7.389787 S    T        2025.03.08
0D00:00:00.175220678 MSFT 288  3.557074 S    T        2025.03.08
0D00:00:00.182020766 JPM  101  7.154379 B    T        2025.03.08
0D00:00:00.236256852 JPM  508  9.207793 B    N        2025.03.08
0D00:00:00.245483237 JPM  177  9.338188 B    N        2025.03.08
0D00:00:00.262837919 GE   879  6.18538  B    L        2025.03.08
0D00:00:00.267904380 MSFT 131  9.724386 B    N        2025.03.08
0D00:00:00.275574979 JPM  395  7.547554 S    T        2025.03.08
0D00:00:00.277730277 JPM  676  2.215053 B    L        2025.03.08
0D00:00:00.284491743 JPM  467  3.585405 S    N        2025.03.08
0D00:00:00.285450072 GE  

In [168]:
//your answer her
parse "update date:.z.d from trade"
5#trade
![trade;();0b;(enlist `date)!(enlist `.z.d)]

!
`trade
()
0b
(,`date)!,`.z.d
time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.004375310 JPM  273  2.612327 B    N       
0D00:00:00.046089480 GE   16   8.989    S    N       
0D00:00:00.144635306 MSFT 57   5.550662 B    N       
0D00:00:00.165940544 JPM  606  7.389787 S    T       
0D00:00:00.175220678 MSFT 288  3.557074 S    T       
time                 sym  size price    side exchange date      
----------------------------------------------------------------
0D00:00:00.004375310 JPM  273  2.612327 B    N        2025.03.08
0D00:00:00.046089480 GE   16   8.989    S    N        2025.03.08
0D00:00:00.144635306 MSFT 57   5.550662 B    N        2025.03.08
0D00:00:00.165940544 JPM  606  7.389787 S    T        2025.03.08
0D00:00:00.175220678 MSFT 288  3.557074 S    T        2025.03.08
0D00:00:00.182020766 JPM  101  7.154379 B    T        2025.03.08
0D00:00:00.236256852 JPM  508  9.207793 B    N        2025.03.08
0D00:0

# Functional `delete` 
The last case to consider is functional [`delete`](https://code.kx.com/q/basics/funsql/#delete) which is really a  special version of `update`, similar to how `exec` is related to `select`. 

We already know that `delete` is more limited than `update` - we can only remove whole rows or columns, since we cannot accept a partial row or column. The way in which we indicate we are deleting, rather that updating is the format of the aggregation value. 

In [171]:
parse"delete time from trade" //this is an empty symbol list

!
`trade
()
0b
,,`time


With `update`, the aggregation value was a dictionary, but for `delete` this is a symbol list corresponding to the columns to be deleted.

In [172]:
(delete time from trade)~![trade;();0b;enlist `time]
![trade;();0b;enlist `time]

1b
sym  size price    side exchange
--------------------------------
JPM  273  2.612327 B    N       
GE   16   8.989    S    N       
MSFT 57   5.550662 B    N       
JPM  606  7.389787 S    T       
MSFT 288  3.557074 S    T       
JPM  101  7.154379 B    T       
JPM  508  9.207793 B    N       
JPM  177  9.338188 B    N       
GE   879  6.18538  B    L       
MSFT 131  9.724386 B    N       
JPM  395  7.547554 S    T       
JPM  676  2.215053 B    L       
JPM  467  3.585405 S    N       
GE   999  9.465283 S    N       
GE   371  3.626004 S    T       
JPM  453  9.554201 B    N       
JPM  798  10.72355 S    N       
GE   757  6.345187 B    T       
GE   882  4.040299 S    L       
MSFT 321  2.552804 B    N       
..


If we are in fact deleting rows rather than columns, we can provide an empty symbol list and use our constraints or where clause to specify the rows: 

In [173]:
![trade;enlist(=;`side;enlist `B);0b;`$()]~delete from trade where side =`B
![trade;enlist(=;`side;enlist `B);0b;`$()]

1b
time                 sym  size price    side exchange
-----------------------------------------------------
0D00:00:00.046089480 GE   16   8.989    S    N       
0D00:00:00.165940544 JPM  606  7.389787 S    T       
0D00:00:00.175220678 MSFT 288  3.557074 S    T       
0D00:00:00.275574979 JPM  395  7.547554 S    T       
0D00:00:00.284491743 JPM  467  3.585405 S    N       
0D00:00:00.285450072 GE   999  9.465283 S    N       
0D00:00:00.295165534 GE   371  3.626004 S    T       
0D00:00:00.323367932 JPM  798  10.72355 S    N       
0D00:00:00.351501211 GE   882  4.040299 S    L       
0D00:00:00.447388955 JPM  200  4.059909 S    L       
0D00:00:00.461190422 MSFT 512  4.720506 S    N       
0D00:00:00.475220054 MSFT 199  8.938436 S    N       
0D00:00:00.612842625 GE   786  10.31797 S    N       
0D00:00:00.636878639 MSFT 556  2.356244 S    T       
0D00:00:00.675553587 JPM  412  4.528074 S    N       
0D00:00:00.694200694 GE   73   3.233227 S    N       
0D00:00:00.855985526 MSFT

As stated above, we cannot delete the fractions of rows or columns - functionally or otherwise! 

In [175]:
//delete time from trade where sym =`JPM

##### Exercise

In one statement, delete all rows from the `trade` table where the side was sell, or the exchange was N.

In [ ]:
//this one is tricky, so lets start with our qSQL! 
delete from trade where (side =`S) or exchange =`N //alternative: delete from trade where (side =`S)|exchange =`N

In [ ]:
//now that we have that, lets parse it
parse"delete from trade where (side =`S) or exchange =`N" //our "or" here makes this just one constraint

In [ ]:
c: enlist (or;(=;`side;enlist `S);(=;`exchange;enlist `N)) //our constraint 
![trade;c;0b;`$()]  //delete is straightforward once we have our constraint

In [179]:
// Write your code here
parse "delete from trade where (side=`S) or exchange=`N"
![trade; enlist(|;(=;`side;enlist`S);(=;`exchange;enlist`N));0b;`symbol$()]

!
`trade
,,(|;(=;`side;,`S);(=;`exchange;,`N))
0b
`symbol$()
time                 sym  size price     side exchange
------------------------------------------------------
0D00:00:00.182020766 JPM  101  7.154379  B    T       
0D00:00:00.262837919 GE   879  6.18538   B    L       
0D00:00:00.277730277 JPM  676  2.215053  B    L       
0D00:00:00.341562921 GE   757  6.345187  B    T       
0D00:00:00.487194983 GE   73   10.43362  B    T       
0D00:00:00.559789508 MSFT 169  6.31297   B    T       
0D00:00:00.570159504 MSFT 625  0.4931005 B    L       
0D00:00:00.679208768 GE   633  9.520934  B    L       
0D00:00:00.696681865 JPM  584  9.595467  B    L       
0D00:00:00.751196438 MSFT 18   4.34695   B    L       
0D00:00:00.756182324 JPM  338  2.839943  B    L       
0D00:00:00.806247032 MSFT 226  10.3794   B    L       
0D00:00:00.968378051 GE   586  2.890916  B    L       
0D00:00:01.145559297 MSFT 772  2.606172  B    L       
0D00:00:01.189870016 GE   873  9.477372  B    T       
0D00

##### Quiz Time!
Try the Exercises to test your functional statements knowledge!